In [1]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install transformers
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 24.5 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 244.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 252.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 207.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 224.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 371.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 153.3 MB/s eta 0:00:00
  Attempting uninstall: numpy━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  6/23 [tensorboard-data-server]
    Found existing installation: numpy 2.2.6━━━━━━━━━━━━━━━━━━  6/23 [tensorboard-data-server]
    Uninstalling numpy-2.2.6:91m╸━━━━━━━━━━━━━━━━━━━━━━━━  9/23 [numpy]d-data-server]
      Successfully uninstalled numpy-2.2.6━━━━━━━━━━━━━━━━━━━━━━━━  9/23 [numpy]
   ━━━

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers import DefaultDataCollator

2025-06-06 18:54:40.123097: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 18:54:40.129874: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 18:54:40.142310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749236080.162396     172 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749236080.168220     172 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749236080.184339     172 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
# 1. Load IMDB dataset from TFDS
(ds_train, ds_test), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

2025-06-06 18:54:45.511747: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# 2. Load DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [5]:
# 3. Tokenization helper
def encode(text, label):
    tokens = tokenizer(
        text.numpy().decode('utf-8'),
        truncation=True,
        padding='max_length',
        max_length=256
    )
    return tokens['input_ids'], tokens['attention_mask'], label

def tf_encode(text, label):
    input_ids, attention_mask, label = tf.py_function(
        encode, [text, label], [tf.int32, tf.int32, tf.int64]
    )
    input_ids.set_shape([256])
    attention_mask.set_shape([256])
    label.set_shape([])
    return {"input_ids": input_ids, "attention_mask": attention_mask}, label

In [6]:
# 4. Prepare dataset pipeline
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 16

ds_train = ds_train.map(tf_encode, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.shuffle(2048).batch(BATCH_SIZE).prefetch(AUTOTUNE)

ds_test = ds_test.map(tf_encode, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [7]:
# 5. Load DistilBERT model for sequence classification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [8]:
# 6. Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [9]:
# 7. Train
model.fit(ds_train, epochs=2, validation_data=ds_test)

Epoch 1/2


2025-06-06 18:55:02.807383: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


1563/1563 [==============================] - 13378s 9s/step - loss: 0.2848 - accuracy: 0.8808 - val_loss: 0.2352 - val_accuracy: 0.9019
Epoch 2/2
1563/1563 [==============================] - 13543s 9s/step - loss: 0.1602 - accuracy: 0.9413 - val_loss: 0.2432 - val_accuracy: 0.9079


In [10]:
# 8. Evaluate
results = model.evaluate(ds_test)
print(f"\nTest accuracy: {results[1]:.4f}")

1563/1563 [==============================] - 2913s 2s/step - loss: 0.2432 - accuracy: 0.9079

Test accuracy: 0.9079


In [12]:
# 9. Save model if desired
model.save_pretrained('./sentiment_model_distilbert')